In [26]:
import pickle
import json

In [27]:
class args :
    def __init__(self):
        self.Full_data_path=r'../data/download_data/kgdata.pkl'
        self.node_type='gene/protein'
        self.Task_data_path = '../ref_projects/LukePi/data/SL'
        self.cv = 'CV3'
        self.n_fold = 5

args=args()

In [28]:
with open (args.Full_data_path,'rb') as f:
    kgdata=pickle.load(f)
with open("../data/download_data/node_index_dic.json",'rb') as f:
    node_index=json.load(f)

In [29]:
print(kgdata.keys())
print(kgdata)

['edge_index', 'x']
HeteroData(
  anatomy={ x=[14035, 1] },
  biological_process={ x=[28642, 1] },
  cellular_component={ x=[4176, 1] },
  disease={ x=[17080, 1] },
  drug={ x=[7957, 1] },
  effect/phenotype={ x=[15311, 1] },
  exposure={ x=[818, 1] },
  gene/protein={ x=[27671, 1] },
  molecular_function={ x=[11169, 1] },
  pathway={ x=[2516, 1] },
  (disease, disease_phenotype_negative, effect/phenotype)={ edge_index=[2, 1365] },
  (molecular_function, molfunc_protein, gene/protein)={ edge_index=[2, 69530] },
  (drug, off-label use, disease)={ edge_index=[2, 2730] },
  (drug, indication, disease)={ edge_index=[2, 9600] },
  (drug, drug_effect, effect/phenotype)={ edge_index=[2, 64784] },
  (molecular_function, molfunc_molfunc, molecular_function)={ edge_index=[2, 27148] },
  (pathway, pathway_protein, gene/protein)={ edge_index=[2, 42646] },
  (pathway, pathway_pathway, pathway)={ edge_index=[2, 5070] },
  (anatomy, anatomy_anatomy, anatomy)={ edge_index=[2, 28064] },
  (anatomy, ana

In [30]:
import argparse
import json
import logging
from time import time
import os
import torch_geometric.transforms as T
from MyLoader import HeteroDataset
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import numpy as np
from model import HGT
import pandas as pd

import pickle
import math
from torch_geometric.datasets import OGB_MAG
import torch.nn.init as init
from sklearn.metrics import f1_score, roc_auc_score,auc,balanced_accuracy_score,cohen_kappa_score,precision_recall_curve

In [32]:
from torch_geometric.loader import HGTLoader

def Downstream_data_preprocess(args,cv,n_fold,node_type_dict): #FIXME
    """
    load SL data and preprocess before training 
    """
    task_data_path=args.Task_data_path
    train_data=pd.read_csv(f"{task_data_path}/{cv}/cv_{n_fold}/train.txt",header=None,sep=' ')
    test_data=pd.read_csv(f"{task_data_path}/{cv}/cv_{n_fold}/test.txt",header=None,sep=' ')
   
    test_data.columns=[0,1,2]
    train_data[0]=train_data[0].astype(str).map(node_type_dict)
    train_data[1]=train_data[1].astype(str).map(node_type_dict)
    test_data[0]=test_data[0].astype(str).map(node_type_dict)
    test_data[1]=test_data[1].astype(str).map(node_type_dict)
    train_data=train_data.dropna()
    test_data=test_data.dropna()
    train_data[0]=train_data[0].astype(int)
    train_data[1]=train_data[1].astype(int)
    test_data[0]=test_data[0].astype(int)
    test_data[1]=test_data[1].astype(int)
    # low data scenario settings
    if args.do_low_data:
        num_sample=int(train_data.shape[0]*args.train_data_ratio)
        print(num_sample)
        train_data=train_data.sample(num_sample,replace=False,random_state=0)
        train_data.reset_index(inplace=True)
        print(f'train_data.size:{train_data.shape[0]}')

    train_node=list(set(train_data[0])|set(train_data[1]))
    train_mask=torch.zeros((27671))
    test_mask=torch.zeros((27671))
    test_node=list(set(test_data[0])|set(test_data[1]))
    train_mask[train_node]=1
    test_mask[test_node]=1
    train_mask=train_mask.bool()
    test_mask=test_mask.bool()
    num_train_node=len(train_node)
    num_test_node=len(test_node)
    return train_data,test_data,train_mask,test_mask,num_train_node,num_test_node

def Construct_loader(args,kgdata,train_mask,test_mask,node_type,num_train_node,num_test_node):
    """
    construct loader for train/test data
    """

    train_loader = HGTLoader(kgdata,
    num_samples={key: [args.sample_nodes] * args.sample_layers for key in kgdata.node_types},shuffle=False,
    batch_size=num_train_node,
    input_nodes=(node_type,train_mask),num_workers=args.num_workers)

    test_loader=HGTLoader(kgdata,
    num_samples={key: [args.sample_nodes] * args.sample_layers for key in kgdata.node_types},
    batch_size=num_test_node,
    input_nodes=(node_type,test_mask),num_workers=args.num_workers,shuffle=False)

    return train_loader,test_loader


n_fold = args.n_fold
train_data,test_data,train_mask,test_mask,num_train_node,num_test_node=Downstream_data_preprocess(args,args.cv,n_fold,gene_protein)
train_loader, test_loader = Construct_loader(args,kgdata,train_mask,test_mask,args.node_type,num_train_node,num_test_node)

NameError: name 'gene_protein' is not defined

In [ ]:
node_type=args.node_type
num_nodes_type=len(kgdata.node_types)
num_edge_type=len(kgdata.edge_types)
num_nodes=kgdata.num_nodes

In [ ]:
kgdata.node_types

['anatomy',
 'biological_process',
 'cellular_component',
 'disease',
 'drug',
 'effect/phenotype',
 'exposure',
 'gene/protein',
 'molecular_function',
 'pathway']

In [ ]:
num_nodes=kgdata.num_nodes
num_nodes

129375

In [ ]:
input_node_embeddings = torch.nn.Embedding(num_nodes_type, 16)

In [ ]:
input_node_embeddings.weight.data

tensor([[ 0.2647, -0.3318,  1.9300,  0.7090,  2.1468,  1.0788, -1.8217,  0.5937,
          1.3203,  0.3222,  1.2115, -0.1300,  0.1351, -2.3575,  0.1373, -0.9029],
        [ 2.6803, -1.6097, -0.1909, -1.0081, -0.2722,  0.3071, -0.6171, -1.1328,
         -1.3235,  0.5680, -0.0482,  0.9665,  0.5694,  2.0087, -1.2142,  0.4647],
        [-1.2270,  0.3460,  0.6839,  0.0995,  0.5282, -0.8753, -0.2060, -1.4988,
          0.2550, -0.1892, -0.9877,  0.1544,  1.5109,  1.2141,  0.3656,  0.6832],
        [ 1.0723, -1.2680,  1.5123, -0.3494,  1.1303,  0.2936, -1.0086, -0.2305,
         -0.0867,  0.5867,  2.3677,  1.6656,  0.3009,  1.7255,  0.8373, -0.6350],
        [ 0.7550, -0.2788,  0.1110,  1.6777,  0.8265,  0.9138,  0.6897,  0.3184,
         -1.6997, -2.0051, -0.2839, -0.4002,  0.7139,  0.9077,  0.3302,  0.9257],
        [ 0.3193,  2.9194,  0.3326, -0.1390,  0.8928, -0.3686, -0.3867,  0.4544,
         -0.5156, -1.0124, -0.8428,  1.1027,  0.9954,  1.3821,  1.3825, -1.4479],
        [ 0.3182,  1.7

In [ ]:
torch.nn.init.xavier_uniform_(input_node_embeddings.weight.data)

tensor([[ 0.3677,  0.1961, -0.4575, -0.2868, -0.3501, -0.2493, -0.3145,  0.4797,
          0.0637, -0.2231,  0.4210,  0.3329, -0.3840,  0.3078,  0.0989,  0.1364],
        [-0.4410,  0.1073,  0.4762,  0.2389,  0.4147,  0.3244,  0.4693, -0.2145,
          0.2293,  0.0307,  0.3301,  0.4745, -0.0357,  0.4445, -0.3164, -0.1970],
        [-0.2649,  0.2021, -0.3940,  0.4260, -0.0607,  0.4677, -0.3306,  0.0612,
          0.3478,  0.3496, -0.0325,  0.0309,  0.2178,  0.0632,  0.1648,  0.4714],
        [ 0.4461, -0.1188,  0.3893,  0.4462,  0.3637,  0.0942,  0.2324, -0.1691,
          0.2501, -0.3223, -0.3623,  0.4676,  0.4427,  0.1891, -0.1469,  0.0237],
        [ 0.2260,  0.2596,  0.0363,  0.1346,  0.2434, -0.2289,  0.2005,  0.1280,
         -0.0460, -0.3925, -0.1939,  0.3543, -0.3781, -0.2190,  0.0548,  0.2837],
        [ 0.3399,  0.0881, -0.0727,  0.0144, -0.1689, -0.0682,  0.3673,  0.3920,
         -0.1178,  0.1036, -0.2327, -0.2228, -0.3812, -0.1952,  0.2823, -0.4644],
        [ 0.4180, -0.2

In [ ]:
for i in range(len(kgdata.node_types)):
    num_repeat=kgdata[kgdata.node_types[i]].x.shape[0]
    kgdata[kgdata.node_types[i]].x =input_node_embeddings(torch.tensor(i)).repeat([num_repeat,1]).detach()